In [1]:
import pandas as pd
import re
import os

def read_table_from_file(file_path):
    """
    从文件中读取表格数据
    支持Excel(.xlsx)和CSV(.csv)格式
    """
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        return pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    else:
        raise ValueError("不支持的文件格式，请使用Excel(.xlsx/.xls)或CSV(.csv)文件")

def clean_text(text):
    """
    清理文本，移除空格和特殊字符以便更好地匹配
    """
    if pd.isna(text):
        return ""
    # 移除所有空格和特殊字符，只保留字母和数字
    return re.sub(r'[^a-zA-Z0-9]', '', str(text).lower())

def match_tables(df1, df2):
    """
    匹配两个表格的数据
    """
    # 在表2中创建新列来存储匹配的PY值
    df2['PY'] = None
    
    # 预处理表1的processed_text，创建清理后的版本
    df1['cleaned_text'] = df1['processed_text'].apply(clean_text)
    
    # 进行匹配
    for idx2, row2 in df2.iterrows():
        document_text = clean_text(row2['Document'])
        matched_pys = []
        
        for idx1, row1 in df1.iterrows():
            processed_text = row1['cleaned_text']
            # 检查清理后的processed_text是否在清理后的Document中
            if processed_text and processed_text in document_text:
                matched_pys.append(row1['PY'])
        
        # 如果找到匹配，将PY值添加到表2
        if matched_pys:
            # 这里取第一个匹配的PY，如果需要所有匹配的PY，可以改为 ','.join(map(str, matched_pys))
            df2.at[idx2, 'PY'] = matched_pys[0]
    
    # 移除临时列
    df1.drop('cleaned_text', axis=1, inplace=True)
    
    return df2

def main():
    # 使用原始字符串或正斜杠来处理文件路径
    # 方法1: 使用原始字符串 (在字符串前加r)
    file1_path = r"C:\Users\apple\Downloads\agetest\files\table1.xlsx"  # 请替换为您的表1文件路径
    file2_path = r"C:\Users\apple\Downloads\agetest\files\table2.xlsx"  # 请替换为您的表2文件路径
    output_path = r"matched_result.xlsx"  # 输出文件路径
    
    # 方法2: 使用正斜杠
    # file1_path = "C:/path/to/table1.xlsx"  # 使用正斜杠
    
    # 方法3: 使用双反斜杠
    # file1_path = "C:\\path\\to\\table1.xlsx"  # 使用双反斜杠
    
    try:
        # 读取表格数据
        print("正在读取表格数据...")
        df1 = read_table_from_file(file1_path)
        df2 = read_table_from_file(file2_path)
        
        print(f"表1数据行数: {len(df1)}")
        print(f"表2数据行数: {len(df2)}")
        
        # 显示前几行数据以便确认
        print("\n表1前几行数据:")
        print(df1.head())
        print("\n表2前几行数据:")
        print(df2.head())
        
        # 进行匹配
        print("\n正在进行数据匹配...")
        result_df = match_tables(df1, df2)
        
        # 显示匹配结果
        print("\n匹配结果:")
        print(result_df)
        
        # 保存结果到文件
        if output_path.endswith('.xlsx') or output_path.endswith('.xls'):
            result_df.to_excel(output_path, index=False)
        else:
            result_df.to_csv(output_path, index=False)
        
        print(f"\n匹配完成！结果已保存到: {output_path}")
        
        # 显示匹配统计
        matched_count = result_df['PY'].notna().sum()
        print(f"成功匹配的行数: {matched_count}/{len(result_df)}")
        
    except FileNotFoundError as e:
        print(f"文件未找到: {e}")
        print("请检查文件路径是否正确")
    except Exception as e:
        print(f"处理过程中出现错误: {e}")

# 如果直接运行此脚本
if __name__ == "__main__":
    main()

正在读取表格数据...
表1数据行数: 3517
表2数据行数: 2417

表1前几行数据:
     PY                   UT  \
0  2025  WOS:001511045000002   
1  2025  WOS:001502499300004   
2  2025  WOS:001494090700025   
3  2025  WOS:001504041300003   
4  2025  WOS:001510927700002   

                                      processed_text  
0  idiopathic normal pressure hydrocephalus sulca...  
1  air quality healthy ageing predictive modeling...  
2  efficient fpga implementation multiplication o...  
3  estimation prediction com terrain feature embe...  
4  egofall real-time privacy-preserving fall risk...  

表2前几行数据:
                                            Document  Topic
0  idiopathic normal pressure hydrocephalus sulca...      1
1  air quality healthy ageing predictive modeling...     26
2  estimation prediction com terrain feature embe...     23
3  language-agnostic automated assessment listene...     10
4  impact gait parameter variability fall risk as...      0

正在进行数据匹配...

匹配结果:
                                       

In [4]:
import pandas as pd
import numpy as np
import os
import pymannkendall as mk
from scipy import stats

def count_publications_by_topic_and_year(file_path):
    """
    统计每个Topic在2015-2024年每年的发文量
    """
    # 读取Excel文件
    df = pd.read_excel(file_path)
    
    # 过滤掉2025年的数据，只保留2015-2024年
    df = df[(df['PY'] >= 2015) & (df['PY'] <= 2024)]
    
    # 统计每个Topic每年的发文量
    result = df.groupby(['Topic', 'PY']).size().reset_index(name='Count')
    
    # 创建透视表，Topic为行，年份为列
    pivot_table = result.pivot_table(
        index='Topic', 
        columns='PY', 
        values='Count', 
        fill_value=0
    )
    
    # 确保包含2015-2024所有年份的列
    all_years = list(range(2015, 2025))
    for year in all_years:
        if year not in pivot_table.columns:
            pivot_table[year] = 0
    
    # 按年份排序列
    pivot_table = pivot_table.reindex(columns=sorted(pivot_table.columns))
    
    # 计算每个Topic的总发文量
    pivot_table['Total'] = pivot_table.sum(axis=1)
    
    # 按总发文量降序排列
    pivot_table = pivot_table.sort_values('Total', ascending=False)
    
    return pivot_table

def perform_mann_kendall_test(statistics_df):
    """
    对每个Topic的时间序列进行Mann-Kendall趋势检验
    """
    # 提取年份列（2015-2024）
    year_columns = [col for col in statistics_df.columns if isinstance(col, int) and 2015 <= col <= 2024]
    
    # 准备存储结果的列表
    results = []
    
    # 对每个Topic进行Mann-Kendall检验
    for topic, row in statistics_df.iterrows():
        # 提取该Topic的时间序列数据
        time_series = row[year_columns].values
        
        # 只有当时间序列有变化时才进行检验（避免全零序列）
        if np.any(time_series != time_series[0]) and len(np.unique(time_series)) > 1:
            try:
                # 执行Mann-Kendall检验
                mk_result = mk.original_test(time_series)
                
                results.append({
                    'Topic': topic,
                    'Trend': mk_result.trend,
                    'H': mk_result.h,
                    'p': mk_result.p,
                    'Z': mk_result.z,
                    'Tau': mk_result.Tau,
                    's': mk_result.s,
                    'var_s': mk_result.var_s,
                    'slope': mk_result.slope
                })
            except Exception as e:
                # 如果检验失败，记录错误信息
                results.append({
                    'Topic': topic,
                    'Trend': 'Error',
                    'H': False,
                    'p': np.nan,
                    'Z': np.nan,
                    'Tau': np.nan,
                    's': np.nan,
                    'var_s': np.nan,
                    'slope': np.nan,
                    'Error': str(e)
                })
        else:
            # 对于全零或没有变化的序列
            results.append({
                'Topic': topic,
                'Trend': 'No trend',
                'H': False,
                'p': 1.0,
                'Z': 0,
                'Tau': 0,
                's': 0,
                'var_s': 0,
                'slope': 0
            })
    
    # 创建结果DataFrame
    mk_results_df = pd.DataFrame(results)
    mk_results_df.set_index('Topic', inplace=True)
    
    return mk_results_df

def create_comprehensive_report(statistics_df, mk_results_df, output_path):
    """
    创建包含统计数据和Mann-Kendall检验结果的综合报告
    """
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # 1. 基本统计数据
        statistics_df.to_excel(writer, sheet_name='Topic_Year_Statistics')
        
        # 2. Mann-Kendall检验结果
        mk_results_df.to_excel(writer, sheet_name='Mann_Kendall_Test')
        
        # 3. 合并结果（基本统计 + 趋势分析）
        merged_df = statistics_df.copy()
        mk_columns = ['Trend', 'p', 'slope']  # 选择最重要的几列
        for col in mk_columns:
            if col in mk_results_df.columns:
                merged_df[col] = mk_results_df[col]
        
        merged_df.to_excel(writer, sheet_name='Combined_Analysis')
        
        # 4. 趋势统计汇总
        trend_summary = mk_results_df['Trend'].value_counts().reset_index()
        trend_summary.columns = ['Trend', 'Count']
        trend_summary['Percentage'] = (trend_summary['Count'] / len(mk_results_df) * 100).round(2)
        trend_summary.to_excel(writer, sheet_name='Trend_Summary', index=False)
        
        # 5. 显著趋势的Topic（p < 0.05）
        significant_trends = mk_results_df[mk_results_df['p'] < 0.05]
        significant_trends = significant_trends.sort_values('p')
        significant_trends.to_excel(writer, sheet_name='Significant_Trends')

def main():
    """
    主函数：执行完整的统计分析流程
    """
    # 使用安全的文件路径
    #current_dir = os.path.dirname(os.path.abspath(__file__))
    #input_file = os.path.join(current_dir, "主题年份.xlsx")
    #output_file = os.path.join(current_dir, "topic_trend_analysis.xlsx")
    input_file = r"C:\Users\apple\Downloads\agetest\files\topic_year.xlsx"
    output_file = r"C:\Users\apple\Downloads\agetest\files\topic_year_statistics.xlsx"
    
    try:
        print("正在进行Topic年度发文量统计...")
        
        # 检查文件是否存在
        if not os.path.exists(input_file):
            print(f"错误: 找不到文件 {input_file}")
            print("请确保文件与Python脚本在同一目录下")
            return
        
        # 1. 统计每个Topic每年的发文量
        statistics_df = count_publications_by_topic_and_year(input_file)
        print(f"统计完成！共分析 {len(statistics_df)} 个Topic")
        
        # 2. 执行Mann-Kendall趋势检验
        print("正在进行Mann-Kendall趋势检验...")
        mk_results_df = perform_mann_kendall_test(statistics_df)
        print("趋势检验完成！")
        
        # 3. 创建综合报告
        create_comprehensive_report(statistics_df, mk_results_df, output_file)
        
        # 4. 输出关键结果
        print(f"\n=== 分析完成 ===")
        print(f"结果文件: {output_file}")
        
        # 趋势分布统计
        trend_counts = mk_results_df['Trend'].value_counts()
        print(f"\n趋势分布:")
        for trend, count in trend_counts.items():
            percentage = (count / len(mk_results_df) * 100)
            print(f"  {trend}: {count}个Topic ({percentage:.1f}%)")
        
        # 显著趋势统计
        significant_count = len(mk_results_df[mk_results_df['p'] < 0.05])
        print(f"\n显著趋势 (p < 0.05): {significant_count}个Topic ({significant_count/len(mk_results_df)*100:.1f}%)")
        
        # 显示具有显著上升趋势的前5个Topic
        rising_trends = mk_results_df[
            (mk_results_df['Trend'] == 'increasing') & 
            (mk_results_df['p'] < 0.05)
        ].sort_values('slope', ascending=False)
        
        if len(rising_trends) > 0:
            print(f"\n上升趋势最强的5个Topic:")
            for i, (topic, row) in enumerate(rising_trends.head(5).iterrows(), 1):
                print(f"  {i}. Topic {topic}: 斜率={row['slope']:.3f}, p值={row['p']:.4f}")
        
        # 显示具有显著下降趋势的前5个Topic
        falling_trends = mk_results_df[
            (mk_results_df['Trend'] == 'decreasing') & 
            (mk_results_df['p'] < 0.05)
        ].sort_values('slope')
        
        if len(falling_trends) > 0:
            print(f"\n下降趋势最强的5个Topic:")
            for i, (topic, row) in enumerate(falling_trends.head(5).iterrows(), 1):
                print(f"  {i}. Topic {topic}: 斜率={row['slope']:.3f}, p值={row['p']:.4f}")
                
    except Exception as e:
        print(f"处理过程中出现错误: {e}")
        print("请确保已安装所需的库: pip install pandas openpyxl pymannkendall")

# 运行主函数
if __name__ == "__main__":
    main()

正在进行Topic年度发文量统计...
统计完成！共分析 30 个Topic
正在进行Mann-Kendall趋势检验...
趋势检验完成！

=== 分析完成 ===
结果文件: C:\Users\apple\Downloads\agetest\files\topic_year_statistics.xlsx

趋势分布:
  increasing: 27个Topic (90.0%)
  no trend: 3个Topic (10.0%)

显著趋势 (p < 0.05): 27个Topic (90.0%)

上升趋势最强的5个Topic:
  1. Topic 0: 斜率=8.667, p值=0.0002
  2. Topic 2: 斜率=6.000, p值=0.0001
  3. Topic 1: 斜率=5.000, p值=0.0007
  4. Topic 3: 斜率=4.333, p值=0.0200
  5. Topic 4: 斜率=2.600, p值=0.0017
